This is a simple and straightforward solution that assigns schedule to each traffic light that has incomming streets and allocates exactly one second to each incomming street.

In [ ]:
import numpy as np
import math

inputfile = '/kaggle/input/hashcode-2021-oqr-extension/hashcode.in'
outputfile = 'submission.csv'

alpha = 4

input = open(inputfile, 'r')
line = input.readline()
settings = line.split()
duration = int(settings[0])
intersections = int(settings[1])
streets = int(settings[2])
cars = int(settings[3])
reward = int(settings[4])

#We collect information about incomming and outgoing streets for each intersection in a form of matrix
street_matrix = np.empty(shape=(intersections,intersections),dtype='object')

for i in range(streets):
    line = input.readline()
    street = line.split()
    beginning = int(street[0])
    end = int(street[1])
    name = street[2]
    length = int(street[3])
    street_matrix[beginning,end] = {'name':name, 'length':length}
    #We retain both name and length for each street
    #this version of the solution does not use the length yet, but we wil expand on this later


#We collect information about each car's route in a vector
#this version of the solution does not use the car information yet, but we wil expand on this later
car_matrix = np.empty(shape=(cars),dtype='object')

used_streets = {}

for i in range(cars):
    line = input.readline()
    car = line.split()
    new_car = {'length':int(car[0])}
    for j in range(1, len(car)-1):
        new_car[j-1] = car[j]
        if car[j] not in used_streets.keys():
            used_streets[car[j]] = 1 * (int(car[0]) - (j - 1))
        else:
            used_streets[car[j]] += 1 * (int(car[0]) - (j - 1))
        
    new_car[len(car)-2] = car[len(car)-1]       
    car_matrix[i] = new_car
    
input.close()

We go through the street matrix finding all Traffic lights that have incomming streets and createing a dictionary of schedules for them

In [ ]:
for i in range(intersections):
    for j in range(intersections):
        if street_matrix[i,j]:
            if not (street_matrix[i,j]['name'] in used_streets.keys()):
                street_matrix[i,j] = None
                
number_of_schedules = 0
schedules = list()
for j in range(intersections):
    endings = 0
    sequence = {}
    for i in range(intersections):
        if street_matrix[i,j]:
            endings += 1
            sequence[street_matrix[i,j]['name']] = 1
    if endings > 0:
        total_usage = 0
        for key in sequence:
            total_usage += used_streets[key]
        for key in sequence:
            sequence[key] = math.ceil (alpha *  used_streets[key] / float(total_usage))
        number_of_schedules += 1
        light_shedule = {'id':j}
        light_shedule['length'] = endings
        light_shedule['sequence'] = sequence
        schedules.append(light_shedule)

![](http://)We use the Schedules dictionary to form the output file

In [ ]:
with open(outputfile, 'w') as writefile:
    writefile.write(str(number_of_schedules))
    for i in range(number_of_schedules):
        writefile.write('\n'+ str(schedules[i]['id']))
        writefile.write('\n'+ str(schedules[i]['length']))
        for key in schedules[i]['sequence']:
            writefile.write('\n' + key + ' ' + str(schedules[i]['sequence'][key]))